In [67]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import random

from keras.models import load_model

import warnings
warnings.filterwarnings(action='ignore')

In [68]:
image_path = './images/osmo10.jpg'

In [69]:
import cv2
import numpy as np
from ultralytics import YOLO

from torchvision import transforms


from values import *


MODEL = YOLO("./model/best_ver3.pt")


# 인식된 개체 정보
def inference_image(input_img):
    input_img = cv2.resize(input_img, (640, 640))
    img = transforms.ToTensor()(input_img).unsqueeze(0)

    result = MODEL.predict(img)
    boxes = result[0].boxes
    boxesn = boxes.xyxy

    find_cls = boxes.cls.cpu().detach().numpy()
    fin_conf = boxes.conf.cpu().detach().numpy()
    
    labels_list = []
    
    for cls,(x1, y1, x2, y2) in zip(find_cls, boxesn):
        label_dict = {
            'label': CLASS[int(cls)],
            'x1': int(x1),
            'y1': int(y1),
            'x2': int(x2),
            'y2': int(y2),
            'cx': int((x1+x2)/2),
            'cy': int((y1+y2)/2)
        }
        labels_list.append(label_dict)
    print(labels_list)
    return labels_list

# 불러오려는 사진
img = cv2.imread(image_path)
labels_list = inference_image(img)


0: 640x640 1 3, 1 4, 1 5, 3 arrows, 172.9ms
Speed: 0.0ms preprocess, 172.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


[{'label': 4, 'x1': 418, 'y1': 239, 'x2': 474, 'y2': 288, 'cx': 446, 'cy': 263}, {'label': 5, 'x1': 428, 'y1': 326, 'x2': 483, 'y2': 381, 'cx': 456, 'cy': 353}, {'label': 'down', 'x1': 337, 'y1': 159, 'x2': 383, 'y2': 204, 'cx': 360, 'cy': 181}, {'label': 'down', 'x1': 340, 'y1': 334, 'x2': 410, 'y2': 372, 'cx': 375, 'cy': 353}, {'label': 3, 'x1': 406, 'y1': 161, 'x2': 461, 'y2': 207, 'cx': 433, 'cy': 184}, {'label': 'down', 'x1': 335, 'y1': 246, 'x2': 402, 'y2': 280, 'cx': 369, 'cy': 263}]


In [71]:
sorted_data = sorted(labels_list, key=lambda x: x['cy'])

In [86]:
arroow_result_list = []
number_result_list = []
action_result_list = []

In [87]:
for i in sorted_data:
    print(i)
    input_img = cv2.imread(image_path)
    img = cv2.resize(input_img, (640, 640))

    crop_img = img[int(i['y1']):int(i['y2']), 
                    int(i['x1']):int(i['x2'])]
    
    out = crop_img.copy()
    out = 255 - out
    
    # 48 x 48
    output_img_48 = cv2.resize(out, (48, 48), interpolation = cv2.INTER_AREA)

    # 그레이 스케일 및 이진화
    gray_img = cv2.cvtColor(output_img_48, cv2.COLOR_BGR2GRAY)
    ret, th1 = cv2.threshold(gray_img, 150, 255, cv2.THRESH_BINARY)
    
    X_test = np.array(th1)
    input_shape = 2304
    X_test = X_test / 255
    X_test = X_test.reshape(-1, input_shape)
    
    if (i['label'] == 'up') | (i['label'] == 'down') | (i['label'] == 'left') | (i['label'] == 'right'):
        loaded_model = load_model('./model/arrow_model_01.h5')

        test_pred = loaded_model.predict(X_test)
        for i, a in enumerate(test_pred[0]):
            if a == test_pred.max():
                if i == 0:
                    arroow_result_list.append('d')
                elif i == 1:
                    arroow_result_list.append('l')
                elif i == 2:
                    arroow_result_list.append('r')
                elif i == 3:
                    arroow_result_list.append('u')
        
    else:
        loaded_model = load_model('./model/number_model_01.h5')

        test_pred = loaded_model.predict(X_test)
        for i, a in enumerate(test_pred[0]):
            if a == test_pred.max():
                if i == 0:
                    number_result_list.append(2)
                elif i == 1:
                    number_result_list.append(3)
                elif i == 2:
                    number_result_list.append(4)
                elif i == 3:
                    number_result_list.append(5)
                    

img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

for i in range(len(sorted_data)):
    if i % 2 == 0:
        if type(sorted_data[i]['label']) == str:
            x = int(sorted_data[i]['cx'] + (sorted_data[i]['cx'] - sorted_data[i+1]['cx']) * 2)
            y = int(sorted_data[i]['cy'] + (sorted_data[i]['cy'] - sorted_data[i+1]['cy']) * 2)
            action_pixel = (y, x)
        elif type(sorted_data[i]['label']) == int:
            x = int(sorted_data[i+1]['cx'] + (sorted_data[i+1]['cx'] - sorted_data[i]['cx']) * 2)
            y = int(sorted_data[i+1]['cy'] + (sorted_data[i+1]['cy'] - sorted_data[i]['cy']) * 2)
            action_pixel = (y, x)
            
        if img_hsv[action_pixel][0] < 50:
            action_result_list.append('Run')
        elif img_hsv[action_pixel][0] < 115:
            action_result_list.append('Hand')
        else:
            action_result_list.append('Jump')
        


{'label': 'down', 'x1': 337, 'y1': 159, 'x2': 383, 'y2': 204, 'cx': 360, 'cy': 181}
1/1 [==============================] - 0s 26ms/step
{'label': 3, 'x1': 406, 'y1': 161, 'x2': 461, 'y2': 207, 'cx': 433, 'cy': 184}
1/1 [==============================] - 0s 27ms/step
{'label': 4, 'x1': 418, 'y1': 239, 'x2': 474, 'y2': 288, 'cx': 446, 'cy': 263}
1/1 [==============================] - 0s 25ms/step
{'label': 'down', 'x1': 335, 'y1': 246, 'x2': 402, 'y2': 280, 'cx': 369, 'cy': 263}
1/1 [==============================] - 0s 27ms/step
{'label': 5, 'x1': 428, 'y1': 326, 'x2': 483, 'y2': 381, 'cx': 456, 'cy': 353}
1/1 [==============================] - 0s 26ms/step
{'label': 'down', 'x1': 340, 'y1': 334, 'x2': 410, 'y2': 372, 'cx': 375, 'cy': 353}
1/1 [==============================] - 0s 27ms/step


In [88]:
action_result_list

['Run', 'Jump', 'Hand']

In [89]:
arroow_result_list, number_result_list, action_result_list

(['d', 'l', 'l'], [3, 4, 5], ['Run', 'Jump', 'Hand'])

In [90]:
response_data = {
    "direction": arroow_result_list,
    "number": number_result_list,
    "action": action_result_list
}

In [91]:
response_data

{'direction': ['d', 'l', 'l'],
 'number': [3, 4, 5],
 'action': ['Run', 'Jump', 'Hand']}

In [ ]:
i = 0
while i < len(sorted_data):
    if type(sorted_data[i]['label']) == str:
        x = int(sorted_data[i]['cx'] + (sorted_data[i]['cx'] - sorted_data[i+1]['cx']) * 2)
        y = int(sorted_data[i]['cy'] + (sorted_data[i]['cy'] - sorted_data[i+1]['cy']) * 2)
        action_pixel = (y, x)
    elif type(sorted_data[0]['label']) == int:
        x = int(sorted_data[i+1]['cx'] + (sorted_data[i+1]['cx'] - sorted_data[i]['cx']) * 2)
        y = int(sorted_data[i+1]['cy'] + (sorted_data[i+1]['cy'] - sorted_data[i]['cy']) * 2)
        action_pixel = (y, x)
    action_result_list.append(action_pixel)    
    i += 2

In [ ]:
for i in range(len(sorted_data)):
    if i % 2 == 0:
        if type(sorted_data[i]['label']) == str:
            x = int(sorted_data[i]['cx'] + (sorted_data[i]['cx'] - sorted_data[i+1]['cx']) * 2)
            y = int(sorted_data[i]['cy'] + (sorted_data[i]['cy'] - sorted_data[i+1]['cy']) * 2)
            action_pixel = (y, x)
        elif type(sorted_data[0]['label']) == int:
            x = int(sorted_data[i+1]['cx'] + (sorted_data[i+1]['cx'] - sorted_data[i]['cx']) * 2)
            y = int(sorted_data[i+1]['cy'] + (sorted_data[i+1]['cy'] - sorted_data[i]['cy']) * 2)
            action_pixel = (y, x)
        action_result_list.append(action_pixel)